In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import polars as pl
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from astropy.stats import sigma_clip

In [2]:
DEBUG = False

DATA_ROOT = '../input/'
SPLIT = 'train'

In [3]:
sensor_sizes_dict = {
    "AIRS-CH0": [[11250, 32, 356], [32, 356]],
    "FGS1": [[135000, 32, 32], [32, 32]],
}  # input, mask

In [4]:
adc_info = pd.read_csv(f'{DATA_ROOT}/{SPLIT}_adc_info.csv', index_col='planet_id')
print('Total num of examples:', adc_info.shape[0])

if DEBUG:
    adc_info = adc_info.head(32)
    print('DEBUG, only use 32 examples!')

Total num of examples: 673


In [5]:
axis_info = pd.read_parquet(f'{DATA_ROOT}/axis_info.parquet')

In [6]:
def read_signal_data(planet_id, sensor):
    ''' sensor: AIRS-CH0 or FGS1
    '''
    signal = pd.read_parquet(f'{DATA_ROOT}/{SPLIT}/{planet_id}/{sensor}_signal.parquet', engine='pyarrow')
    dark = pd.read_parquet(f'{DATA_ROOT}/{SPLIT}/{planet_id}/{sensor}_calibration/dark.parquet', engine='pyarrow')
    dead = pd.read_parquet(f'{DATA_ROOT}/{SPLIT}/{planet_id}/{sensor}_calibration/dead.parquet', engine='pyarrow')
    flat = pd.read_parquet(f'{DATA_ROOT}/{SPLIT}/{planet_id}/{sensor}_calibration/flat.parquet', engine='pyarrow')
    linear_corr = pd.read_parquet(f'{DATA_ROOT}/{SPLIT}/{planet_id}/{sensor}_calibration/linear_corr.parquet', engine='pyarrow')
    read = pd.read_parquet(f'{DATA_ROOT}/{SPLIT}/{planet_id}/{sensor}_calibration/read.parquet', engine='pyarrow')

    # reshape
    signal = signal.values.reshape(sensor_sizes_dict[sensor][0]).astype(np.float64)
    dark = dark.values.reshape(sensor_sizes_dict[sensor][1]).astype(np.float64)
    dead = dead.values.reshape(sensor_sizes_dict[sensor][1])  # bool
    flat = flat.values.reshape(sensor_sizes_dict[sensor][1]).astype(np.float64)
    linear_corr = linear_corr.values.reshape([6] + sensor_sizes_dict[sensor][1]).astype(np.float64)
    read = read.values.reshape(sensor_sizes_dict[sensor][1]).astype(np.float64)
    
    return signal, dark, dead, flat, linear_corr, read

In [7]:
planet_id = 785834
sensor = 'AIRS-CH0'
# sensor = 'FGS1'

In [8]:
signal, dark, dead, flat, linear_corr, read = read_signal_data(planet_id, sensor)

In [9]:
signal.shape, signal.dtype

((11250, 32, 356), dtype('float64'))

In [10]:
dark.shape, dark.dtype

((32, 356), dtype('float64'))

In [11]:
dead.shape, dead.dtype

((32, 356), dtype('bool'))

In [12]:
flat.shape, flat.dtype

((32, 356), dtype('float64'))

In [13]:
linear_corr.shape, linear_corr.dtype

((6, 32, 356), dtype('float64'))

In [14]:
read.shape, read.dtype

((32, 356), dtype('float64'))

## Calibration

### Step 1: Analog-to-Digital Conversion

The Analog-to-Digital Conversion (adc) is performed by the detector to convert the pixel voltage into an integer number. We revert this operation by using the gain and offset for the calibration files 'train_adc_info.csv'.

> 模拟到数字转换（adc）由检测器执行，用于将像素电压转换为整数。我们使用增益和偏移量来还原该操作

In [15]:
def ADC_convert(signal, gain, offset):
    signal /= gain
    signal += offset
    return signal

In [16]:
def get_gain_offset(planet_id, sensor):
    gain = adc_info.loc[planet_id][f'{sensor}_adc_gain']
    offset = adc_info.loc[planet_id][f'{sensor}_adc_offset']
    return gain, offset

In [17]:
gain, offset = get_gain_offset(planet_id, sensor)

In [18]:
signal = ADC_convert(signal, gain, offset)

In [19]:
signal.shape

(11250, 32, 356)

### Step 2: Mask hot/dead pixel

The dead pixels map is a map of the pixels that do not respond to light and, thus, can’t be accounted for any calculation. In all these frames the dead pixels are masked using python masked arrays. The bad pixels are thus masked but left uncorrected. Some methods can be used to correct bad-pixels but this task, if needed, is left to the participants.

> 坏像素图是那些对光没有反应的像素的分布图，因此在任何计算中不能考虑这些像素。在所有这些帧中，坏像素都使用Python掩码数组进行了掩盖。这些坏像素因此被掩盖但未被修正。有一些方法可以用来修正坏像素，但这项任务（如果需要）将留给参与者处理。

In [20]:
def mask_hot_dead(signal, dead, dark):
    hot = sigma_clip(dark, sigma=5, maxiters=5).mask
    hot = np.tile(hot, (signal.shape[0], 1, 1))
    dead = np.tile(dead, (signal.shape[0], 1, 1))

    # Set values to np.nan where dead or hot pixels are found
    signal[dead] = np.nan
    signal[hot] = np.nan
    return signal

In [21]:
np.isnan(signal).sum()

0

In [22]:
signal = mask_hot_dead(signal, dead, dark)

In [23]:
np.isnan(signal).sum()

708750

### Step 3: linearity Correction

The non-linearity of the pixels’ response can be explained as capacitive leakage on the readout electronics of each pixel during the integration time. The number of electrons in the well is proportional to the number of photons that hit the pixel, with a quantum efficiency coefficient. However, the response of the pixel is not linear with the number of electrons in the well. This effect can be described by a polynomial function of the number of electrons actually in the well. The data is provided with calibration files linear_corr.parquet that are the coefficients of the inverse polynomial function and can be used to correct this non-linearity effect.

> 像素响应的非线性可以解释为在积分时间内，每个像素的读出电子设备发生的电容泄漏。井中的电子数量与击中像素的光子数量成正比，其中包含一个量子效率系数。然而，像素的响应与井中的电子数量并不是线性关系。这个效应可以用井中实际电子数量的多项式函数来描述。数据提供了校准文件 linear_corr.parquet，这些文件包含了逆多项式函数的系数，可以用于校正这种非线性效应。

In [24]:
def apply_linear_corr(c, signal):
    assert c.shape[0] == 6  # Ensure the polynomial is of degree 5
    return (
        (((c[5] * signal + c[4]) * signal + c[3]) * signal + c[2]) * signal + c[1]
    ) * signal + c[0]

In [25]:
np.nanmax(signal), np.nanmin(signal), np.nanmean(signal)

(70089.18442311407, -778.9165332747325, 786.2558964499108)

In [26]:
signal = apply_linear_corr(linear_corr, signal)

In [27]:
np.nanmax(signal), np.nanmin(signal), np.nanmean(signal)

(73434.73312296864, -1256.0400117285885, 793.5201280040981)

### Step 4: dark current subtraction

The data provided include calibration for dark current estimation, which can be used to pre-process the observations. Dark current represents a constant signal that accumulates in each pixel during the integration time, independent of the incoming light. To obtain the corrected image, the following conventional approach is applied: The data provided include calibration files such as dark frames or dead pixels' maps. They can be used to pre-process the observations. The dark frame is a map of the detector response to a very short exposure time, to correct for the dark current of the detector.
$$\text{image - dark} \times \Delta t $$ 
The corrected image is conventionally obtained via the following: where the dark current map is first corrected for the dead pixel.

> 提供的数据包括用于暗电流估计的校准，这些校准可以用于预处理观测结果。暗电流表示在积分时间内每个像素中积累的恒定信号，与进入的光无关。为了获得校正后的图像，通常采用以下方法：提供的数据包括校准文件，如暗场帧或坏点图。这些文件可以用于预处理观测结果。暗场帧是一张非常短曝光时间的探测器响应图，用于校正探测器的暗电流。校正后的图像通常通过以下方式获得：首先对坏点修正暗电流图，然后通过公式$$\text{image - dark} \times \Delta t $$ 获得校正后的图像。

In [28]:
def clean_dark(signal, dark, dt):
    dark = np.tile(dark, (signal.shape[0], 1, 1))
    signal -= dark * dt[:, np.newaxis, np.newaxis]
    return signal

In [29]:
dt_airs = axis_info["AIRS-CH0-integration_time"].dropna().values
dt_fgs1 = np.ones(sensor_sizes_dict['FGS1'][0][0]) * 0.1

In [30]:
dt = dt_airs if sensor == 'AIRS-CH0' else dt_fgs1
dt[1::2] += 0.1

In [31]:
signal = clean_dark(signal, dark, dt)

### Step 5: Get Correlated Double Sampling (CDS)

The science frames are alternating between the start of the exposure and the end of the exposure. The lecture scheme is a ramp with a double sampling, called Correlated Double Sampling (CDS), the detector is read twice, once at the start of the exposure and once at the end of the exposure. The final CDS is the difference (End of exposure) - (Start of exposure).

> 科学帧在曝光的开始和结束时交替进行。读取方案是带有双重采样的斜坡，这被称为相关双重采样（Correlated Double Sampling, CDS）。探测器在曝光开始和结束时各读取一次。最终的CDS值是曝光结束时的读取值减去曝光开始时的读取值。

In [32]:
def get_cds(signal):
    cds = signal[1::2, :, :] - signal[::2, :, :]
    return cds

In [33]:
signal.shape

(11250, 32, 356)

In [34]:
signal = get_cds(signal)

In [35]:
signal.shape

(5625, 32, 356)

### Step 6: Flat Field Correction

The flat field is a map of the detector response to uniform illumination, to correct for the pixel-to-pixel variations of the detector, for example the different quantum efficiencies of each pixel.

> 平场图是探测器对均匀照明响应的映射，用于校正探测器中像素与像素之间的变异，例如每个像素的不同量子效率。

In [36]:
def correct_flat_field(flat, signal):
    return signal / flat

In [37]:
signal = correct_flat_field(flat, signal)

In [38]:
signal.shape

(5625, 32, 356)

### 组合起来

In [39]:
def preprocess_signal_data(planet_id, reduction=True):
    
    dt_airs = axis_info["AIRS-CH0-integration_time"].dropna().values
    dt_fgs1 = np.ones(sensor_sizes_dict['FGS1'][0][0]) * 0.1
    
    res = []
    for sensor in ["FGS1", "AIRS-CH0"]:
        signal, dark, dead, flat, linear_corr, read = read_signal_data(planet_id, sensor)
        
        # step1: ADC
        gain, offset = get_gain_offset(planet_id, sensor)
        signal = ADC_convert(signal, gain, offset)
        
        # step2: Mask hot/dead pixel
        signal = mask_hot_dead(signal, dead, dark)
        
        # step3: linearity Correction
        signal = apply_linear_corr(linear_corr, signal)
        
        # step4: dark current subtraction
        dt = dt_airs if sensor == 'AIRS-CH0' else dt_fgs1
        dt[1::2] += 0.1
        signal = clean_dark(signal, dark, dt)
        
        # step5: CDS
        signal = get_cds(signal)
        
        # step6: Flat Field Correction
        signal = correct_flat_field(flat, signal)

        # step7: Mean reduction
        if reduction:
            if sensor == 'AIRS-CH0':
                signal = np.nanmean(signal, axis=1).astype(np.float32)
            else:
                signal = np.nanmean(signal, axis=(1, 2)).astype(np.float32)
        
        res.append(signal)
    
    return res  # FGS1, AIRS-CH0

## Process

In [40]:
import os
from tqdm.auto import tqdm
from concurrent.futures import ProcessPoolExecutor

In [41]:
os.mkdir('train_processed')

In [42]:
def task(planet_id):
    fgs1_signal, airs_signal = preprocess_signal_data(planet_id)
    np.savez(f'train_processed/{planet_id}.npz', fgs1_signal=fgs1_signal, airs_signal=airs_signal)

In [43]:
with ProcessPoolExecutor(32) as exe:
    list( tqdm(exe.map(task, adc_info.index.tolist()), total=adc_info.shape[0]) )

  0%|          | 0/673 [00:00<?, ?it/s]

In [46]:
sensor_sizes_dict

{'AIRS-CH0': [[11250, 32, 356], [32, 356]],
 'FGS1': [[135000, 32, 32], [32, 32]]}

In [47]:
(adc_info.shape[0], sensor_sizes_dict['FGS1'][0][0] // 2, *sensor_sizes_dict['FGS1'][0][1:])

(673, 67500, 32, 32)

In [49]:
fgs1_signal.shape

(67500,)

In [50]:
airs_signal.shape

(5625, 356)

In [51]:
f_raw_train = []
a_raw_train = []
for planet_id in tqdm(adc_info.index):
    signal_data = np.load(f'train_processed/{planet_id}.npz')
    fgs1_signal = signal_data['fgs1_signal']
    airs_signal = signal_data['airs_signal']
    f_raw_train.append(fgs1_signal)
    a_raw_train.append(airs_signal)

  0%|          | 0/673 [00:00<?, ?it/s]

In [64]:
f_raw_train = np.stack(f_raw_train, axis=0)
f_raw_train.shape

(673, 67500)

In [65]:
a_raw_train = np.stack(a_raw_train, axis=0)
a_raw_train.shape

(673, 5625, 356)

In [66]:
np.savez('train_processed.npz', f_raw_train=f_raw_train, a_raw_train=a_raw_train)